## 类定义

类提供了把数据和功能绑定在一起的方法。创建新类时创建了新的对象 类型，从而能够创建该类型的新 实例。实例具有能维持自身状态的属性，还具有能修改自身状态的方法（由其所属的类来定义）。

和其他编程语言相比，Python 的类只使用了很少的新语法和语义。Python 的类有点类似于 C++ 和 Modula-3 中类的结合体，而且支持面向对象编程（OOP）的所有标准特性：类的继承机制支持多个基类、派生的类能覆盖基类的方法、类的方法能调用基类中的同名方法。对象可包含任意数量和类型的数据。和模块一样，类也支持 Python 动态特性：在运行时创建，创建后还可以修改。

## 名称和对象

对象之间相互独立，多个名称（甚至是多个作用域内的多个名称）可以绑定到同一对象。这在其他语言中通常被称为别名。Python 初学者通常不容易理解这个概念，处理数字、字符串、元组等不可变基本类型时，可以不必理会。但是，对于涉及可变对象（如列表、字典，以及大多数其他类型）的 Python 代码的语义，别名可能会产生意料之外的效果。这样做，通常是为了让程序受益，因为别名在某些方面就像指针。例如，传递对象的代价很小，因为实现只传递一个指针；如果函数修改了作为参数传递的对象，调用者就可以看到更改——无需像 Pascal 那样用两个不同的机制来传参。

In [1]:
L = [1, 2, 3]
L1 = L
print(L1)
L.append(4)
print(L1)

[1, 2, 3]
[1, 2, 3, 4]


## 作用域和命名空间

### 命名空间

`namespace` （命名空间）是从名称到对象的映射。现在，大多数命名空间都使用 Python 字典实现，但除非涉及到性能优化，我们一般不会关注这方面的事情，而且将来也可能会改变这种方式。命名空间的例子有：内置名称集合（包括 `abs()` 函数以及内置异常的名称等）；一个模块的全局名称；一个函数调用中的局部名称。对象的属性集合也是命名空间的一种形式。关于命名空间的一个重要知识点是，不同命名空间中的名称之间绝对没有关系；例如，两个不同的模块都可以定义 `maximize` 函数，且不会造成混淆。用户使用函数时必须要在函数名前面加上模块名。

点号之后的名称是 属性。例如，表达式 `z.real` 中，`real` 是对象 `z` 的属性。严格来说，对模块中名称的引用是属性引用：表达式 `modname.funcname` 中，`modname` 是模块对象，`funcname` 是模块的属性。模块属性和模块中定义的全局名称之间存在直接的映射：它们共享相同的命名空间！ [1]

属性可以是只读的或者可写的。 在后一种情况下，可以对属性进行赋值。 模块属性是可写的：你可以写入 `modname.the_answer = 42`。 也可以使用 `del` 语句删除可写属性。 例如，`del modname.the_answer` 将从名为 `modname` 对象中移除属性 `the_answer`。

命名空间是在不同时刻创建的，且拥有不同的生命周期。内置名称的命名空间是在 Python 解释器启动时创建的，永远不会被删除。模块的全局命名空间在读取模块定义时创建；通常，模块的命名空间也会持续到解释器退出。从脚本文件读取或交互式读取的，由解释器顶层调用执行的语句是 __main__ 模块调用的一部分，也拥有自己的全局命名空间。内置名称实际上也在模块里，即 `builtins` 。

函数的局部命名空间在函数被调用时被创建，并在函数返回或抛出未在函数内被处理的异常时，被删除。（实际上，用“遗忘”来描述实际发生的情况会更好一些。）当然，每次递归调用都有自己的局部命名空间。

### 作用域

一个命名空间的 作用域 是 Python 代码中的一段文本区域，从这个区域可直接访问该命名空间。“可直接访问”的意思是，该文本区域内的名称在被非限定引用时，查找名称的范围，是包括该命名空间在内的。

作用域虽然是被静态确定的，但会被动态使用。执行期间的任何时刻，都会有 3 或 4 个“命名空间可直接访问”的嵌套作用域：

最内层作用域，包含局部名称，并首先在其中进行搜索

那些外层闭包函数的作用域，包含“非局部、非全局”的名称，从最靠内层的那个作用域开始，逐层向外搜索。

倒数第二层作用域，包含当前模块的全局名称

最外层（最后搜索）的作用域，是内置名称的命名空间

如果一个名称被声明为全局，则所有引用和赋值都将直接指向“倒数第二层作用域”，即包含模块的全局名称的作用域。 要重新绑定在最内层作用域以外找到的变量，可以使用 `nonlocal` 语句；如果未使用 `nonlocal` 声明，这些变量将为只读（尝试写入这样的变量将在最内层作用域中创建一个 新的 局部变量，而使得同名的外部变量保持不变）。

通常，当前局部作用域将（按字面文本）引用当前函数的局部名称。在函数之外，局部作用域引用与全局作用域一致的命名空间：模块的命名空间。 类定义在局部命名空间内再放置另一个命名空间。

划重点，作用域是按字面文本确定的：模块内定义的函数的全局作用域就是该模块的命名空间，无论该函数从什么地方或以什么别名被调用。另一方面，实际的名称搜索是在运行时动态完成的。但是，Python 正在朝着“编译时静态名称解析”的方向发展，因此不要过于依赖动态名称解析！（局部变量已经是被静态确定了。）

Python 有一个特殊规定。如果不存在生效的 `global` 或 `nonlocal` 语句，则对名称的赋值总是会进入最内层作用域。赋值不会复制数据，只是将名称绑定到对象。删除也是如此：语句 del x 从局部作用域引用的命名空间中移除对 x 的绑定。所有引入新名称的操作都是使用局部作用域：尤其是 `import` 语句和函数定义会在局部作用域中绑定模块或函数名称。

`global` 语句用于表明特定变量在全局作用域里，并应在全局作用域中重新绑定；`nonlocal` 语句表明特定变量在外层作用域中，并应在外层作用域中重新绑定。

In [2]:



def scope_test():
    def do_local():
        spam = "local spam"
    
    def do_nonlocal():
        nonlocal spam
        spam = "nonlocal spam"
    
    def do_global():
        global spam
        spam = "global spam"
    
    spam = "test spam"
    do_local()
    print("After local assignment", spam)
    do_nonlocal()
    print("After nonlocal assignment", spam)
    do_global()
    print("After global assignment", spam)
    
scope_test()
print("In global scope", spam)
        

After local assignment test spam
After nonlocal assignment nonlocal spam
After global assignment nonlocal spam
In global scope global spam


## 类定义

最简单的类定义形式如下：

In [ ]:
class ClassName:
    # 语句 1
    # 语句 n
    pass

## 类对象
类对象支持两种操作：属性引用和实例化

In [ ]:
class MyClass:
    """一个简单的示例类"""
    i = 12345
    def f(self):
        return "hello world"
    
print(MyClass.i)
print(MyClass.f)

# 类的 实例化 使用函数表示法。 可以把类对象视为是返回该类的一个新实例的不带参数的函数。
x = MyClass()
print(x)
print(x.f())
print(MyClass.__doc__)

12345
<function MyClass.f at 0x10b03e2a0>
hello world
一个简单的示例类


类的 实例化 使用函数表示法。 可以把类对象视为是返回该类的一个新实例的不带参数的函数。

根据特定初始状态定的对象

In [7]:
class MyClass:
    def __init__(self) -> None:
        self.data = []

x = MyClass()
x.data.append(1)
print(x.data)

[1]


带参数的实例化对象

In [8]:
class Complex:
    def __init__(self, realpart, imagpart) -> None:
        self.realpart = realpart
        self.imagpart = imagpart

x = Complex(3.0, -4.5)
print(x.realpart, x.imagpart)

3.0 -4.5


## 实例对象
实例对象所能理解的唯一操作是属性引用。有两种有效的属性名称，数据属性和方法。
- 数据属性不需要声明，就像局部变量一样，他们将在首次被赋值时产生
- 另一种实例属性是方法，方法是从属于对象的函数。实例对象的有效方法名称依赖与其所属的类。

In [9]:
class MyClass:
    pass

x = MyClass()
x.y = 10
print(x.y)

10


根据定义，一个类中所有是函数对象的属性都定义了其实例相应的方法。因此在示例中 `x.f` 是有效的方法引用，因为 `MyClass.f` 是一个函数。
`x.f` 与 `MyClass.f` 并不是一回事，一个是方法对象一个是函数对象。

### 方法对象

通常方法对象在绑定后立即被调用，但是方法对象并不是必须被立即调用，它可以保存起来以后再调用


In [13]:
class MyClass:
    def f(self):
        return "hello world"
    
x = MyClass()
print(x.f())

xf = x.f
print(xf())
print(MyClass.f(x))

hello world
hello world
hello world


总而言之，方法的运作方式如下。 当一个实例的非数据属性被引用时，将搜索该实例所属的类。 如果名称表示一个属于函数对象的有效类属性，则指向实例对象和函数对象的引用将被打包为一个方法对象。 当传入一个参数列表调用该方法对象时，将基于实例对象和参数列表构造一个新的参数列表，并传入这个新参数列表调用相应的函数对象。


## 类和实例变量
实例变量用于每个实例的唯一数据，类变量用于类的所有实例共享的属性和方法。

In [1]:
class Dog:
    kind = "canine"
    def __init__(self, name) -> None:
        self.name = name

d = Dog("Fido")
e = Dog("Buddy")
print(d.kind, e.kind)
print(d.name, e.name)

canine canine
Fido Buddy


类属性的错误用法

In [2]:
class Dog:
    kind = "canine"
    tricks = []
    def __init__(self, name) -> None:
        self.name = name
    
    def add_trick(self, trick):
        self.tricks.append(trick)
        
d = Dog("Fido")
e = Dog("Buddy")
d.add_trick("roll over")
e.add_trick("play dead")
print(d.tricks, e.tricks)


['roll over', 'play dead'] ['roll over', 'play dead']


同样的名称出现在类和实例中，属性查找会选择实例

In [3]:
class Warehouse:
   purpose = 'storage'
   region = 'west'

w1 = Warehouse()
print(w1.purpose, w1.region)

w2 = Warehouse()
w2.region = 'east'
print(w2.purpose, w2.region, Warehouse.region)

storage west
storage east west


## 继承

派生类定义的执行过程与基类相同。 当构造类对象时，基类会被记住。 此信息将被用来解析属性引用：如果请求的属性在类中找不到，搜索将转往基类中进行查找。 如果基类本身也派生自其他某个类，则此规则将被递归地应用。

继承的语法如下：


In [ ]:
class BaseClassName:
    pass

class DrivedClassName(BaseClassName):
    # 语句 1
    # 语句 n
    pass


派生类的实例化没有任何特殊之处: DerivedClassName() 会创建该类的一个新实例。 方法引用将按以下方式解析：搜索相应的类属性，如有必要将按基类继承链逐步向下查找，如果产生了一个函数对象则方法引用就生效。

派生类可能会重写其基类的方法。 因为方法在调用同一对象的其他方法时没有特殊权限，所以基类方法在尝试调用调用同一基类中定义的另一方法时，可能实际上调用是该基类的派生类中定义的方法。（对 C++ 程序员的提示：Python 中所有的方法实际上都是 virtual 方法。）

In [ ]:
class Anaimal:
    kind = "anaimal"

class Dog(Anaimal):
    kind = "dog"


dog = Dog()
dog.kind = "cat"
print(dog.kind)
print(isinstance(dog, Dog))
print(issubclass(Dog, Anaimal))


cat
True
True


## 多重继承

搜索从父类继承的属性，深度优先、从左到右。

带有多个基类的类定义语句如下所示:

class DerivedClassName(Base1, Base2, Base3):
    <语句-1>
    .
    .
    .
    <语句-N>

## 私有变量

带有一个下划线的名称 (例如 _spam) 应该被当作是 API 的非公有部分 (无论它是函数、方法或是数据成员)。 这应当被视为一个实现细节，可能不经通知即加以改变。

由于存在对于类私有成员的有效使用场景（例如避免名称与子类所定义的名称相冲突），因此存在对此种机制的有限支持，称为 名称改写。 任何形式为 __spam 的标识符（至少带有两个前缀下划线，至多一个后缀下划线）的文本将被替换为 _classname__spam，其中 classname 为去除了前缀下划线的当前类名称。 这种改写不考虑标识符的句法位置，只要它出现在类定义内部就会进行。

名称改写有助于让子类重写方法而不破坏类内方法调用。例如:

In [ ]:
class Mapping:
    def __init__(self, iterable):
        self.items_list = []
        self.__update(iterable)

    def update(self, iterable):
        for item in iterable:
            self.items_list.append(item)

    __update = update   # 原始 update() 方法的私有副本

class MappingSubclass(Mapping):

    def update(self, keys, values):
        # 为 update() 提供了新的签名
        # 但不会破坏 __init__()
        for item in zip(keys, values):
            self.items_list.append(item)

## 数据类 dataclass

In [12]:
from dataclasses import dataclass

@dataclass
class Employee:
    name: str
    dept: str
    salary: int

john = Employee('john', 'computer lab', 1000)
print(john.dept)

computer lab
